In [1]:
import numpy as np
import pandas as pd
import datetime

# Light Rail - Analysis on Series Frequency (GTFS)

## Stop Times
Get the scheduled times for each stop. This will be used as referrence to identify if the light rail that is on time

In [2]:
stops = pd.read_csv('stop_times.txt')

In [3]:
stops.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,timepoint
0,712,07:36:16,07:36:36,8111,1,Well Station Drive,0,0,1
1,712,07:38:20,07:38:40,8109,2,Nullarbor Avenue,0,0,1
2,712,07:39:51,07:40:11,8107,3,Mapleton Avenue,0,0,1
3,712,07:41:45,07:42:05,8105,4,Manning Clark Crescent,0,0,1
4,712,07:43:43,07:43:43,8100,5,Gungahlin Place,0,0,1


## Trips
Used to identify each trip based on `id`

In [4]:
trips = pd.read_csv('trips.txt')

In [5]:
trips.head()

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed
0,NIS,SU,712,Gungahlin Pl,0,3,1007,1,1
1,ACTO001,SU,780,Alinga St,1,3,1003,1,1
2,ACTO001,SU,718,Gungahlin Pl,0,3,1004,1,1
3,ACTO001,SU,784,Alinga St,1,3,1003,1,1
4,ACTO001,SU,722,Gungahlin Pl,0,3,1004,1,1


# Light Rail Data Feed (Realtime)
This section looks at getting the realtime updates from Transport Canberra http://files.transport.act.gov.au/feeds/lightrail.pb

## Building a template data feed
- Initialize the FeedMessage parser from Google
- Get the reponnse from the API
- Pass the response to the parser


In [6]:
from google.transit import gtfs_realtime_pb2
import requests

# initialise the feed message parser from Google
feed = gtfs_realtime_pb2.FeedMessage()

# get the response from the api
response = requests.get('http://files.transport.act.gov.au/feeds/lightrail.pb', allow_redirects=True)

# pass the response to the Parser
feed.ParseFromString(response.content)

# entity attribute contains feed data
len(feed.entity)

74

The dataset contains {{ len(feed.entity) }} light rail trips

In [8]:
trip_updates = [ent for ent in feed.entity if ent.HasField('trip_update')]

In [9]:
trip_id = []
location = []
odometer = []
speed = []
timestamp = []
congestion = []
vehicle_id = []
vehicle_label = []
vehicle_license = []
stop_seq = []

trip_id_r = []
stop_seq_r = []
stop_arrival_time = []
stop_arrival_delay = []
stop_departure_time = []
stop_departure_delay = []
stop_id = []
timestamp_r = []

for trip in feed.entity:
    if trip.HasField('trip_update'):
        trip_id_r.append(trip.trip_update.trip.trip_id)
        stop_seq_r.append(trip.trip_update.stop_time_update[0].stop_sequence)
        stop_arrival_time.append(trip.trip_update.stop_time_update[0].arrival.time)
        stop_arrival_delay.append(trip.trip_update.stop_time_update[0].arrival.delay)
        stop_departure_time.append(trip.trip_update.stop_time_update[0].departure.time)
        stop_departure_delay.append(trip.trip_update.stop_time_update[0].departure.delay)
        stop_id.append(trip.trip_update.stop_time_update[0].stop_id)
        timestamp_r.append(trip.trip_update.timestamp)
    else:
        trip_id.append(trip.vehicle.trip.trip_id)
        location.append([(trip.vehicle.position.latitude),(trip.vehicle.position.longitude)])
        odometer.append(trip.vehicle.position.odometer)
        speed.append(trip.vehicle.position.speed)
        timestamp.append(trip.vehicle.timestamp)
        congestion.append(trip.vehicle.congestion_level)
        vehicle_id.append(trip.vehicle.vehicle.id)
        vehicle_label.append(trip.vehicle.vehicle.label)
        vehicle_license.append(trip.vehicle.vehicle.license_plate)
        stop_seq.append(trip.vehicle.current_stop_sequence)

In [10]:
df = pd.DataFrame([trip_id, location, odometer, speed, timestamp, congestion, vehicle_id, vehicle_label, vehicle_license, stop_seq]).transpose()
df.columns = ['Trip ID', 'Location', 'Odometer', 'Speed', 'Timestamp', 'Congestion', 'Vehicle Id', 'Vehicle Label', 'Vehicle License', 'Stop Sequence']

In [11]:
df

,Trip ID,Location,Odometer,Speed,Timestamp,Congestion,Vehicle Id,Vehicle Label,Vehicle License,Stop Sequence
0,492,"[-35.18699645996094, 149.14352416992188]",1.75743e+07,0,1558681050,1,11,LRV11,LRV11,2
1,,"[-35.277828216552734, 149.12930297851562]",1.21136e+07,0,1558681050,1,2,LRV2,LRV2,0
2,362,"[-35.26640701293945, 149.1311492919922]",1.32598e+07,7.825,1558681050,1,10,LRV10,LRV10,3
3,359,"[-35.18701171875, 149.14324951171875]",1.42591e+07,0,1558681050,1,6,LRV6,LRV6,12
4,489,"[-35.26581573486328, 149.13128662109375]",1.07666e+07,6.34722,1558681050,1,7,LRV7,LRV7,12
5,361,"[-35.24196243286133, 149.13551330566406]",1.38119e+07,15.95,1558681050,1,5,LRV5,LRV5,7
6,491,"[-35.208927154541016, 149.1474151611328]",1.63775e+07,2.50833,1558681050,1,12,LRV12,LRV12,6
7,360,"[-35.221065521240234, 149.14480590820312]",1.94079e+07,13.2583,1558681050,1,9,LRV9,LRV9,9
8,490,"[-35.24370574951172, 149.13475036621094]",8.78886e+06,11.1111,1558681050,1,14,LRV14,LRV14,8


In [12]:
df_realtime = pd.DataFrame([trip_id_r, stop_seq_r, stop_arrival_time, stop_arrival_delay, stop_departure_time, stop_departure_delay, stop_id, timestamp_r]).transpose()
df_realtime.columns = ['Trip ID', 'Stop Sequence', 'Arrival Time', 'Arrival Delay', 'Departure Time', 'Departure Delay', 'Stop ID', 'Timestamp']

In [13]:
df_realtime['Trip ID'].unique()

array(['407', '273', '413', '288', '423', '298', '433', '310', '443',
       '316', '449', '322', '455', '328', '461', '334', '467', '340',
       '473', '346', '483', '358', '493', '368', '505', '377', '515',
       '381', '519', '385', '523', '389', '527', '393', '531', '397',
       '535', '401', '540', '270', '408', '278', '416', '291', '426',
       '301', '437', '312', '445', '318', '451', '324', '457', '330',
       '463', '336', '469', '342', '475', '352', '487', '362', '497',
       '372', '511'], dtype=object)

In [14]:
df_realtime[df_realtime['Trip ID'].isin(list(df['Trip ID']))]

,Trip ID,Stop Sequence,Arrival Time,Arrival Delay,Departure Time,Departure Delay,Stop ID,Timestamp
61,362,3,1558681058,14,1558681078,14,8125,1558681050


In [15]:
df_realtime['Arrival Time'] = pd.to_datetime(df_realtime['Arrival Time'],unit='s')
df_realtime['Departure Time'] = pd.to_datetime(df_realtime['Departure Time'],unit='s')
df_realtime['Timestamp'] = pd.to_datetime(df_realtime['Timestamp'],unit='s')

In [21]:
df_realtime.head(30)

,Trip ID,Stop Sequence,Arrival Time,Arrival Delay,Departure Time,Departure Delay,Stop ID,Timestamp
0,407,13,2019-05-23 20:30:15,375,2019-05-23 20:30:15,375,8129,2019-05-24 06:57:30
1,273,13,2019-05-23 20:58:16,256,2019-05-23 20:58:16,256,8100,2019-05-24 06:57:30
2,413,13,2019-05-23 21:29:30,330,2019-05-23 21:29:30,330,8129,2019-05-24 06:57:30
3,288,13,2019-05-23 21:56:26,146,2019-05-23 21:56:26,146,8100,2019-05-24 06:57:30
4,423,13,2019-05-23 22:28:12,252,2019-05-23 22:28:12,252,8129,2019-05-24 06:57:30
5,298,13,2019-05-23 22:56:19,139,2019-05-23 22:56:19,139,8100,2019-05-24 06:57:30
6,433,13,2019-05-23 23:25:07,67,2019-05-23 23:25:07,67,8129,2019-05-24 06:57:30
7,310,13,2019-05-23 23:54:39,39,2019-05-23 23:54:39,39,8100,2019-05-24 06:57:30
8,443,13,2019-05-24 00:24:57,57,2019-05-24 00:24:57,57,8129,2019-05-24 06:57:30
9,316,13,2019-05-24 00:53:54,-6,2019-05-24 00:53:54,-6,8100,2019-05-24 06:57:30


In [30]:
df_realtime[df_realtime['Trip ID'] == '273']

,Trip ID,Stop Sequence,Arrival Time,Arrival Delay,Departure Time,Departure Delay,Stop ID,Timestamp
1,273,13,2019-05-23 20:58:16,256,2019-05-23 20:58:16,256,8100,2019-05-24 06:57:30


In [29]:
trip_updates[1]

id: "4102048"
trip_update {
  trip {
    trip_id: "273"
  }
  stop_time_update {
    stop_sequence: 13
    arrival {
      delay: 256
      time: 1558645096
    }
    departure {
      delay: 256
      time: 1558645096
    }
    stop_id: "8100"
  }
  timestamp: 1558681050
}

In [31]:
stops[(stops['trip_id'] == 273)]

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,timepoint
4612,273,06:30:00,06:30:00,8129,1,Alinga Street,0,0,1
4613,273,06:31:54,06:32:14,8127,2,Elouera Street,0,0,1
4614,273,06:33:24,06:33:44,8125,3,Ipima Street,0,0,1
4615,273,06:34:45,06:35:05,8123,4,Macarthur Avenue,0,0,1
4616,273,06:36:44,06:37:04,8121,5,Dickson Interchange,0,0,1
4617,273,06:38:41,06:39:01,8119,6,Swinden Street,0,0,1
4618,273,06:40:45,06:41:05,8117,7,Phillip Avenue,0,0,1
4619,273,06:43:00,06:43:20,8115,8,EPIC and Racecourse,0,0,1
4620,273,06:46:33,06:46:53,8111,9,Well Station Drive,0,0,1
4621,273,06:48:37,06:48:57,8109,10,Nullarbor Avenue,0,0,1
